In [ ]:
from cfg import *

In [ ]:
from util.datagen import generator, tumor_generator
from util.model import build_seg_model

import segmentation_models as sm
sm.set_framework('tf.keras')
import tensorflow.keras.backend as k
import gc

from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [3]:
import os, glob
from sklearn.utils import shuffle
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import IPython
from IPython.display import Image, display
from tensorflow.keras.preprocessing.image import load_img, array_to_img
import PIL
from PIL import ImageOps
#os.environ["CUDA_VISIBLE_DEVICES"] = '0'

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
        print(e)
plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['figure.figsize'] = (10,8)

strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

2 Physical GPUs, 2 Logical GPUs
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Number of devices: 2


In [4]:
train_gen = generator(BATCH_SIZE,INPUT_SIZE,TRAIN_ZIP, is_train=True, zero_labels = ZERO_LABELS)
valid_gen = generator(BATCH_SIZE,INPUT_SIZE,VALID_ZIP, is_train=False, zero_labels = ZERO_LABELS)
test_gen = generator(BATCH_SIZE,INPUT_SIZE,TEST_ZIP, is_train=False, zero_labels = ZERO_LABELS)

In [ ]:
batch_x,batch_y = train_gen.__getitem__(2)

for idx in range(0,5):
    plt.figure(figsize = (12,6));
    d = np.unique(batch_x,return_counts=True)
    plt.subplot(1,3,1);plt.imshow((batch_x[idx]))
    plt.subplot(1,3,2);plt.imshow(np.squeeze(batch_y[idx],axis=-1),vmin=0,vmax=2)
    #plt.subplot(1,3,3);plt.bar(d[0],d[1])

---

## ---

### TRAIN BY KERAS-TUNER

from kerastuner import Hyperband, HyperParameter, HyperParameters, HyperModel

def build_model(hp):
    dice_loss = sm.losses.DiceLoss()
    focal_loss = sm.losses.BinaryFocalLoss(alpha=0.25, gamma=6.0)
    total_loss = dice_loss + focal_loss
    metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]
    
    BACKBONE = hp.Choice('backbone',['inceptionresnetv2','inceptionv3'])
    model = sm.Unet(BACKBONE,input_shape=(512,512,3),classes=1,activation='sigmoid',encoder_weights='imagenet')
    '''
    if hp.Choice('optimizer',['adam','sgd'])=='adam':
        model.compile(Adam(lr=hp.Choice('learning_rate',[1e-3,1e-4,1e-5])),total_loss,metrics)
    else:
        model.compile(SGD(lr=hp.Choice('learning_rate',[1e-3,1e-4,1e-5])),total_loss,metrics)
    '''
    model.compile(optimizer = Adam(hp.Float('learning_rate',1e-4,1e-2,sampling='log')),loss=total_loss,metrics=metrics)
    return model

## CALLBACK
model_path='./data/model/seg_tumor_0718.hdf5'
mc = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
es = EarlyStopping(monitor = 'val_loss',patience = 4)
lr = ReduceLROnPlateau(
    monitor='val_loss',  # 검증 손실을 기준으로 callback이 호출됩니다
    factor=0.5,          # callback 호출시 학습률을 1/2로 줄입니다
    patience=2,         # epoch 4 동안 개선되지 않으면 callback이 호출됩니다
)
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)
callbacks_list = [mc,es,lr,ClearTrainingOutput()]

tuner = Hyperband(
    hypermodel=build_model,
    objective="val_loss",
    max_epochs=10,
    factor=3,
    hyperband_iterations=1,
    distribution_strategy=tf.distribute.MirroredStrategy(),
    directory='tumor_segmentation'
)

tuner.search(
    train_gen, 
    validation_data=valid_gen, 
    epochs=10,
    callbacks = callbacks_list
)

tuner.results_summary()

best_hp = tuner.get_best_hyperparameters()[0]


best_hp.values

best_model = tuner.hypermodel.build(best_hp)

model_path = './data/model/seg_tumor_tuner_inceptionv3.hdf5'
mc = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
es = EarlyStopping(monitor = 'val_loss',patience = 8)
lr = ReduceLROnPlateau(
    monitor='val_loss',  # 검증 손실을 기준으로 callback이 호출됩니다
    factor=0.5,          # callback 호출시 학습률을 1/2로 줄입니다
    patience=4,         # epoch 4 동안 개선되지 않으면 callback이 호출됩니다
)
callbacks_list = [mc,es,lr]

history = best_model.fit(
    train_gen,
    validation_data = valid_gen,
    callbacks = callbacks_list,
    epochs=EPOCHS,
    max_queue_size=15,
    use_multiprocessing=True,
    verbose=1
)
best_model.evaluate(test_gen)

---

### TRAIN BY FIT

In [5]:
print(INPUT_SHAPE)
print(EPOCHS)
print(PATIENCE)
print(MODEL_PATH) 
print(MODEL)
print(BACKBONE)
print(WEIGHT)
print(ACTIVATION)
print(INITIAL_LEARNING_RATE)
print(LOSS)
print(OPTIMIZER)
print(METRIC)

(512, 512, 3)
200
10
./model/0824_deeplabv3.hdf5
deeplab
xception
pascal_voc
sigmoid
0.005
dice+focal
adam
['iou', 'f1-score']


In [6]:
MODEL = 'deeplab'
MODEL_PATH = './model/0825_deeplab_xception.hdf5'
BACKBONE = 'xception'
WEIGHT = 'pascal_voc'
LOSS = 'focal_dice'
N_CLASSES = 1

In [7]:
from util.model import *

In [8]:
from tensorflow.keras.callbacks import Callback
#loss = build_binary_loss(LOSS)
dice_loss = sm.losses.DiceLoss()

# default focal_loss (alpha=0.25, gamma=2.0)
focal_loss = sm.losses.BinaryFocalLoss(alpha=0.25, gamma=6.0)
loss = dice_loss + focal_loss
optim = Adam(INITIAL_LEARNING_RATE)
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]
#metrics = [my_metric]
class ClearMemory(Callback):
    def on_epoch_end(self,epoch,logs=None):
        gc.collect()
        k.clear_session()
        
mc = ModelCheckpoint(filepath=MODEL_PATH, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
es = EarlyStopping(monitor = 'val_loss',patience = 10)
lr = ReduceLROnPlateau(
    monitor='val_loss',  # 검증 손실을 기준으로 callback이 호출됩니다
    factor=0.5,          # callback 호출시 학습률을 1/2로 줄입니다
    patience=5,         # epoch 4 동안 개선되지 않으면 callback이 호출됩니다
)
callback_list = [mc,es,lr,ClearMemory()]

In [9]:
## MULTIGPU
import tensorflow as tf
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
with strategy.scope():
    model = build_seg_model(model = MODEL,backbone=BACKBONE,weight = WEIGHT,input_shape=INPUT_SHAPE,n_classes=N_CLASSES,loss=LOSS,init_lr=INITIAL_LEARNING_RATE,optimizer=OPTIMIZER,is_train=False)
    #model = sm.Deeplabv3(weights = WEIGHT,input_shape=INPUT_SHAPE,classes = N_CLASSES,activation=ACTIVATION,backbone=BACKBONE)
    #model = sm.Unet(BACKBONE, input_shape = INPUT_SHAPE,classes=N_CLASSES, activation=ACTIVATION,encoder_weights=WEIGHT)
    model.compile(optim, loss, metrics)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Number of devices: 2
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:

In [ ]:
callback_list = build_callback(MODEL_PATH,PATIENCE)

In [ ]:
model.fit(train_gen, 
            epochs = EPOCHS,
            validation_data=valid_gen,
            callbacks=callback_list,
            max_queue_size=20,
            workers=10,
            #use_multiprocessing=True,
            #verbose=1
           )

Epoch 1/200
INFO:tensorflow:batch_all_reduce: 440 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 440 all-reduces with algorithm = nccl, num_packs = 1
211/211 [==============================] - 889s 4s/step - loss: 0.6582 - iou_score: 0.3007 - f1-score: 0.4434 - val_loss: 1.2075 - val_iou_score: 0.0132 - val_f1-score: 0.0132

Epoch 00001: val_loss improved from inf to 1.20746, saving model to ./model/0825_deeplab_xception.hdf5


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/200
INFO:tensorflow:batch_all_reduce: 440 all-reduces with algorithm = nccl, num_packs = 1
211/211 [==============================] - 533s 2s/step - loss: 0.4134 - iou_score: 0.5190 - f1-score: 0.6733 - val_loss: 1.8136 - val_iou_score: 0.1290 - val_f1-score: 0.2244

Epoch 00002: val_loss did not improve from 1.20746
Epoch 3/200
211/211 [==============================] - 371s 2s/step - loss: 0.3663 - iou_score: 0.5659 - f1-score: 0.7142 - val_loss: 1.1632 - val_iou_score: 0.0997 - val_f1-score: 0.1786

Epoch 00003: val_loss improved from 1.20746 to 1.16322, saving model to ./model/0825_deeplab_xception.hdf5
Epoch 4/200
211/211 [==============================] - 355s 2s/step - loss: 0.3455 - iou_score: 0.5877 - f1-score: 0.7317 - val_loss: 1.1844 - val_iou_score: 0.1171 - val_f1-score: 0.2060

Epoch 00004: val_loss did not improve from 1.16322
Epoch 5/200
211/211 [==============================] - 365s 2s/step - loss: 0.3259 - iou_score: 0.6068 - f1-score: 0.7485 - val_loss: 1.6

192/211 [==========================>...] - ETA: 1:03 - loss: 0.1914 - iou_score: 0.7483 - f1-score: 0.8524

----

In [ ]:
from tensorflow.keras.callbacks import Callback
#loss = build_binary_loss(LOSS)
dice_loss = sm.losses.DiceLoss()

# default focal_loss (alpha=0.25, gamma=2.0)
focal_loss = sm.losses.BinaryFocalLoss(alpha=0.25, gamma=6.0)
loss = dice_loss + focal_loss
optim = Adam(INITIAL_LEARNING_RATE)
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]
#metrics = [my_metric]
class ClearMemory(Callback):
    def on_epoch_end(self,epoch,logs=None):
        gc.collect()
        k.clear_session()
        
mc = ModelCheckpoint(filepath=MODEL_PATH, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
es = EarlyStopping(monitor = 'val_loss',patience = 10)
lr = ReduceLROnPlateau(
    monitor='val_loss',  # 검증 손실을 기준으로 callback이 호출됩니다
    factor=0.5,          # callback 호출시 학습률을 1/2로 줄입니다
    patience=5,         # epoch 4 동안 개선되지 않으면 callback이 호출됩니다
)
callbacks_list = [mc,es,lr,ClearMemory()]

In [ ]:
## MULTIGPU
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
with strategy.scope():
    #model = build_seg_model(model = MODEL)
    #model = sm.Deeplabv3(weights = WEIGHT,input_shape=INPUT_SHAPE,classes = N_CLASSES,activation=ACTIVATION,backbone=BACKBONE)
    model = sm.Unet(BACKBONE, input_shape = INPUT_SHAPE,classes=N_CLASSES, activation=ACTIVATION,encoder_weights=WEIGHT)
    model.compile(optim, loss, metrics)

In [ ]:
model.fit(train_gen, 
            epochs = EPOCHS,
            validation_data=valid_gen,
            callbacks=callbacks_list,
            max_queue_size=20,
            workers=10,
            #use_multiprocessing=True,
            #verbose=1
           )

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (12,10))
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validate'], loc='upper left')
plt.show()

In [ ]:
e_val = model.evaluate(test_gen)
print(f'IOU Score : {e_val[1]:.2f} F1-Score : {e_val[2]:.2f}')

In [ ]:
#model2 = build_seg_model(model='unet',backbone='inceptionv3',weight = WEIGHT,input_shape=INPUT_SHAPE,n_classes=N_CLASSES,loss=LOSS,init_lr=INITIAL_LEARNING_RATE,optimizer=OPTIMIZER,)
model2 = sm.Unet('inceptionv3',input_shape = INPUT_SHAPE,)
model2.load_weights('./model/0824_unet_inceptionv3.hdf5')
model3 = sm.Unet('efficientnetb0',input_shape = INPUT_SHAPE)
model3.load_weights('./model/0824_unet_eff0.hdf5')
model4 = sm.Deeplabv3(backbone='xception',classes=N_CLASSES,activation=ACTIVATION)
model4.load_weights('./model/0817_deeplabv3_xception_pascal_hsv.hdf5')

In [ ]:
import numpy as np
batch_x,batch_y=  test_gen.__getitem__(2)
for i in range(10,20):

    img,msk = batch_x[i],batch_y[i]
    img = np.expand_dims(img,axis=0)
    pr_msk = model.predict(img)
    pr_msk = np.where(np.squeeze(pr_msk)>0.5,np.squeeze(pr_msk),0)
    pr_msk_2 = model2.predict(img)
    pr_msk_2 = np.where(np.squeeze(pr_msk_2)>0.5,np.squeeze(pr_msk_2),0)
    pr_msk_3 = model3.predict(img)
    pr_msk_3 = np.where(np.squeeze(pr_msk_3)>0.5,np.squeeze(pr_msk_3),0)
    pr_msk_4 = model4.predict(img)
    pr_msk_4 = np.where(np.squeeze(pr_msk_4)>0.5,np.squeeze(pr_msk_4),0)
    plt.figure(figsize = (14,10))
    plt.subplot(1,6,1); plt.title('new'); plt.imshow(pr_msk)
    plt.subplot(1,6,2); plt.title('unet_inception'); plt.imshow(pr_msk_2)
    plt.subplot(1,6,3); plt.title('unet_eff'); plt.imshow(pr_msk_3)
    plt.subplot(1,6,4); plt.title('deeplab_x'); plt.imshow(pr_msk_4)
    plt.subplot(1,6,5); plt.title('true_mask'); plt.imshow(np.squeeze(batch_y[i]))
    plt.subplot(1,6,6); plt.title('true_img'); plt.imshow(np.squeeze((255*img).astype(np.uint8)))

#pr_msk = np.expand_dims(pr_msk,axis=-1)


---

In [ ]:
def SepConv_BN(x, filters, prefix, stride=1, kernel_size=3, rate=1, depth_activation=False, epsilon=1e-3):
    """ SepConv with BN between depthwise & pointwise. Optionally add activation after BN
        Implements right "same" padding for even kernel sizes
        Args:
            x: input tensor
            filters: num of filters in pointwise convolution
            prefix: prefix before name
            stride: stride at depthwise conv
            kernel_size: kernel size for depthwise convolution
            rate: atrous rate for depthwise convolution
            depth_activation: flag to use activation between depthwise & poinwise convs
            epsilon: epsilon to use in BN layer
    """

    if stride == 1:
        depth_padding = 'same'
    else:
        kernel_size_effective = kernel_size + (kernel_size - 1) * (rate - 1)
        pad_total = kernel_size_effective - 1
        pad_beg = pad_total // 2
        pad_end = pad_total - pad_beg
        x = ZeroPadding2D((pad_beg, pad_end))(x)
        depth_padding = 'valid'

    if not depth_activation:
        x = Activation(tf.nn.relu)(x)
    x = DepthwiseConv2D((kernel_size, kernel_size), strides=(stride, stride), dilation_rate=(rate, rate),
                        kernel_initializer = 'he_normal',
                        padding=depth_padding, use_bias=False, name=prefix + '_depthwise')(x)
    x = BatchNormalization(name=prefix + '_depthwise_BN', epsilon=epsilon)(x)
    if depth_activation:
        x = Activation(tf.nn.relu)(x)
    x = Conv2D(filters, (1, 1), padding='same',kernel_initializer = 'he_normal',
               use_bias=False, name=prefix + '_pointwise')(x)
    x = BatchNormalization(name=prefix + '_pointwise_BN', epsilon=epsilon)(x)
    if depth_activation:
        x = Activation(tf.nn.relu)(x)

    return x


def _conv2d_same(x, filters, prefix, stride=1, kernel_size=3, rate=1):
    """Implements right 'same' padding for even kernel sizes
        Without this there is a 1 pixel drift when stride = 2
        Args:
            x: input tensor
            filters: num of filters in pointwise convolution
            prefix: prefix before name
            stride: stride at depthwise conv
            kernel_size: kernel size for depthwise convolution
            rate: atrous rate for depthwise convolution
    """
    if stride == 1:
        return Conv2D(filters,
                      (kernel_size, kernel_size),
                      strides=(stride, stride),
                      padding='same', use_bias=False,
                      dilation_rate=(rate, rate),
                      kernel_initializer = 'he_normal',
                      name=prefix)(x)
    else:
        kernel_size_effective = kernel_size + (kernel_size - 1) * (rate - 1)
        pad_total = kernel_size_effective - 1
        pad_beg = pad_total // 2
        pad_end = pad_total - pad_beg
        x = ZeroPadding2D((pad_beg, pad_end))(x)
        return Conv2D(filters,
                      (kernel_size, kernel_size),
                      strides=(stride, stride),
                      kernel_initializer = 'he_normal',
                      padding='valid', use_bias=False,
                      dilation_rate=(rate, rate),
                      name=prefix)(x)


def _xception_block(inputs, depth_list, prefix, skip_connection_type, stride,
                    rate=1, depth_activation=False, return_skip=False):
    """ Basic building block of modified Xception network
        Args:
            inputs: input tensor
            depth_list: number of filters in each SepConv layer. len(depth_list) == 3
            prefix: prefix before name
            skip_connection_type: one of {'conv','sum','none'}
            stride: stride at last depthwise conv
            rate: atrous rate for depthwise convolution
            depth_activation: flag to use activation between depthwise & pointwise convs
            return_skip: flag to return additional tensor after 2 SepConvs for decoder
            """
    residual = inputs
    for i in range(3):
        residual = SepConv_BN(residual,
                              depth_list[i],
                              prefix + '_separable_conv{}'.format(i + 1),
                              stride=stride if i == 2 else 1,
                              rate=rate,
                              depth_activation=depth_activation)
        if i == 1:
            skip = residual
    if skip_connection_type == 'conv':
        shortcut = _conv2d_same(inputs, depth_list[-1], prefix + '_shortcut',
                                kernel_size=1,
                                stride=stride)
        shortcut = BatchNormalization(name=prefix + '_shortcut_BN')(shortcut)
        outputs = Add()([residual, shortcut])
    elif skip_connection_type == 'sum':
        outputs = Add()([residual, inputs])
    elif skip_connection_type == 'none':
        outputs = residual
    if return_skip:
        return outputs, skip
    else:
        return outputs


def _make_divisible(v, divisor, min_value=None):
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


def _inverted_res_block(inputs, expansion, stride, alpha, filters, block_id, skip_connection, rate=1):
    in_channels = inputs.shape[-1]  # inputs._keras_shape[-1]
    pointwise_conv_filters = int(filters * alpha)
    pointwise_filters = _make_divisible(pointwise_conv_filters, 8)
    x = inputs
    prefix = 'expanded_conv_{}_'.format(block_id)
    if block_id:
        # Expand

        x = Conv2D(expansion * in_channels, kernel_size=1, padding='same',
                   use_bias=False, activation=None,
                   kernel_initializer = 'he_normal',
                   name=prefix + 'expand')(x)
        x = BatchNormalization(epsilon=1e-3, momentum=0.999,
                               name=prefix + 'expand_BN')(x)
        x = Activation(tf.nn.relu6, name=prefix + 'expand_relu')(x)
    else:
        prefix = 'expanded_conv_'
    # Depthwise
    x = DepthwiseConv2D(kernel_size=3, strides=stride, activation=None,kernel_initializer = 'he_normal',
                        use_bias=False, padding='same', dilation_rate=(rate, rate),
                        name=prefix + 'depthwise')(x)
    x = BatchNormalization(epsilon=1e-3, momentum=0.999,
                           name=prefix + 'depthwise_BN')(x)

    x = Activation(tf.nn.relu6, name=prefix + 'depthwise_relu')(x)

    # Project
    x = Conv2D(pointwise_filters,kernel_initializer = 'he_normal',
               kernel_size=1, padding='same', use_bias=False, activation=None,
               name=prefix + 'project')(x)
    x = BatchNormalization(epsilon=1e-3, momentum=0.999,
                           name=prefix + 'project_BN')(x)

    if skip_connection:
        return Add(name=prefix + 'add')([inputs, x])

    # if in_channels == pointwise_filters and stride == 1:
    #    return Add(name='res_connect_' + str(block_id))([inputs, x])

    return x


def Deeplabv3(weights='pascal_voc', input_tensor=None, input_shape=(512, 512, 3), classes=21, backbone='mobilenetv2',
              OS=16, alpha=1., activation=None):
    """ Instantiates the Deeplabv3+ architecture
    Optionally loads weights pre-trained
    on PASCAL VOC or Cityscapes. This model is available for TensorFlow only.
    # Arguments
        weights: one of 'pascal_voc' (pre-trained on pascal voc),
            'cityscapes' (pre-trained on cityscape) or None (random initialization)
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: shape of input image. format HxWxC
            PASCAL VOC model was trained on (512,512,3) images. None is allowed as shape/width
        classes: number of desired classes. PASCAL VOC has 21 classes, Cityscapes has 19 classes.
            If number of classes not aligned with the weights used, last layer is initialized randomly
        backbone: backbone to use. one of {'xception','mobilenetv2'}
        activation: optional activation to add to the top of the network.
            One of 'softmax', 'sigmoid' or None
        OS: determines input_shape/feature_extractor_output ratio. One of {8,16}.
            Used only for xception backbone.
        alpha: controls the width of the MobileNetV2 network. This is known as the
            width multiplier in the MobileNetV2 paper.
                - If `alpha` < 1.0, proportionally decreases the number
                    of filters in each layer.
                - If `alpha` > 1.0, proportionally increases the number
                    of filters in each layer.
                - If `alpha` = 1, default number of filters from the paper
                    are used at each layer.
            Used only for mobilenetv2 backbone. Pretrained is only available for alpha=1.
    # Returns
        A Keras model instance.
    # Raises
        RuntimeError: If attempting to run this model with a
            backend that does not support separable convolutions.
        ValueError: in case of invalid argument for `weights` or `backbone`
    """

    if not (weights in {'pascal_voc', 'cityscapes', None}):
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization), `pascal_voc`, or `cityscapes` '
                         '(pre-trained on PASCAL VOC)')

    if not (backbone in {'xception', 'mobilenetv2'}):
        raise ValueError('The `backbone` argument should be either '
                         '`xception`  or `mobilenetv2` ')

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        img_input = input_tensor

    if backbone == 'xception':
        if OS == 8:
            entry_block3_stride = 1
            middle_block_rate = 2  # ! Not mentioned in paper, but required
            exit_block_rates = (2, 4)
            atrous_rates = (12, 24, 36)
        else:
            entry_block3_stride = 2
            middle_block_rate = 1
            exit_block_rates = (1, 2)
            atrous_rates = (6, 12, 18)

        x = Conv2D(32, (3, 3), strides=(2, 2),
                   name='entry_flow_conv1_1', use_bias=False, padding='same')(img_input)
        x = BatchNormalization(name='entry_flow_conv1_1_BN')(x)
        x = Activation(tf.nn.relu)(x)

        x = _conv2d_same(x, 64, 'entry_flow_conv1_2', kernel_size=3, stride=1)
        x = BatchNormalization(name='entry_flow_conv1_2_BN')(x)
        x = Activation(tf.nn.relu)(x)

        x = _xception_block(x, [128, 128, 128], 'entry_flow_block1',
                            skip_connection_type='conv', stride=2,
                            depth_activation=False)
        x, skip1 = _xception_block(x, [256, 256, 256], 'entry_flow_block2',
                                   skip_connection_type='conv', stride=2,
                                   depth_activation=False, return_skip=True)

        x = _xception_block(x, [728, 728, 728], 'entry_flow_block3',
                            skip_connection_type='conv', stride=entry_block3_stride,
                            depth_activation=False)
        for i in range(16):
            x = _xception_block(x, [728, 728, 728], 'middle_flow_unit_{}'.format(i + 1),
                                skip_connection_type='sum', stride=1, rate=middle_block_rate,
                                depth_activation=False)

        x = _xception_block(x, [728, 1024, 1024], 'exit_flow_block1',
                            skip_connection_type='conv', stride=1, rate=exit_block_rates[0],
                            depth_activation=False)
        x = _xception_block(x, [1536, 1536, 2048], 'exit_flow_block2',
                            skip_connection_type='none', stride=1, rate=exit_block_rates[1],
                            depth_activation=True)

    else:
        OS = 8
        first_block_filters = _make_divisible(32 * alpha, 8)
        x = Conv2D(first_block_filters,
                   kernel_size=3,
                   strides=(2, 2), padding='same', use_bias=False,
                   name='Conv' if input_shape[2] == 3 else 'Conv_')(img_input)
        x = BatchNormalization(
            epsilon=1e-3, momentum=0.999, name='Conv_BN')(x)
        x = Activation(tf.nn.relu6, name='Conv_Relu6')(x)

        x = _inverted_res_block(x, filters=16, alpha=alpha, stride=1,
                                expansion=1, block_id=0, skip_connection=False)

        x = _inverted_res_block(x, filters=24, alpha=alpha, stride=2,
                                expansion=6, block_id=1, skip_connection=False)
        x = _inverted_res_block(x, filters=24, alpha=alpha, stride=1,
                                expansion=6, block_id=2, skip_connection=True)

        x = _inverted_res_block(x, filters=32, alpha=alpha, stride=2,
                                expansion=6, block_id=3, skip_connection=False)
        x = _inverted_res_block(x, filters=32, alpha=alpha, stride=1,
                                expansion=6, block_id=4, skip_connection=True)
        x = _inverted_res_block(x, filters=32, alpha=alpha, stride=1,
                                expansion=6, block_id=5, skip_connection=True)

        # stride in block 6 changed from 2 -> 1, so we need to use rate = 2
        x = _inverted_res_block(x, filters=64, alpha=alpha, stride=1,  # 1!
                                expansion=6, block_id=6, skip_connection=False)
        x = _inverted_res_block(x, filters=64, alpha=alpha, stride=1, rate=2,
                                expansion=6, block_id=7, skip_connection=True)
        x = _inverted_res_block(x, filters=64, alpha=alpha, stride=1, rate=2,
                                expansion=6, block_id=8, skip_connection=True)
        x = _inverted_res_block(x, filters=64, alpha=alpha, stride=1, rate=2,
                                expansion=6, block_id=9, skip_connection=True)

        x = _inverted_res_block(x, filters=96, alpha=alpha, stride=1, rate=2,
                                expansion=6, block_id=10, skip_connection=False)
        x = _inverted_res_block(x, filters=96, alpha=alpha, stride=1, rate=2,
                                expansion=6, block_id=11, skip_connection=True)
        x = _inverted_res_block(x, filters=96, alpha=alpha, stride=1, rate=2,
                                expansion=6, block_id=12, skip_connection=True)

        x = _inverted_res_block(x, filters=160, alpha=alpha, stride=1, rate=2,  # 1!
                                expansion=6, block_id=13, skip_connection=False)
        x = _inverted_res_block(x, filters=160, alpha=alpha, stride=1, rate=4,
                                expansion=6, block_id=14, skip_connection=True)
        x = _inverted_res_block(x, filters=160, alpha=alpha, stride=1, rate=4,
                                expansion=6, block_id=15, skip_connection=True)

        x = _inverted_res_block(x, filters=320, alpha=alpha, stride=1, rate=4,
                                expansion=6, block_id=16, skip_connection=False)

    # end of feature extractor

    # branching for Atrous Spatial Pyramid Pooling

    # Image Feature branch
    shape_before = tf.shape(x)
    b4 = GlobalAveragePooling2D()(x)
    b4_shape = tf.keras.backend.int_shape(b4)
    # from (b_size, channels)->(b_size, 1, 1, channels)
    b4 = Reshape((1, 1, b4_shape[1]))(b4)
    b4 = Conv2D(256, (1, 1), padding='same',
                use_bias=False, name='image_pooling')(b4)
    b4 = BatchNormalization(name='image_pooling_BN', epsilon=1e-5)(b4)
    b4 = Activation(tf.nn.relu)(b4)
    # upsample. have to use compat because of the option align_corners
    size_before = tf.keras.backend.int_shape(x)
    b4 = tf.keras.layers.experimental.preprocessing.Resizing(
            *size_before[1:3], interpolation="bilinear"
        )(b4)
    # simple 1x1
    b0 = Conv2D(256, (1, 1), padding='same', use_bias=False, name='aspp0')(x)
    b0 = BatchNormalization(name='aspp0_BN', epsilon=1e-5)(b0)
    b0 = Activation(tf.nn.relu, name='aspp0_activation')(b0)

    # there are only 2 branches in mobilenetV2. not sure why
    if backbone == 'xception':
        # rate = 6 (12)
        b1 = SepConv_BN(x, 256, 'aspp1',
                        rate=atrous_rates[0], depth_activation=True, epsilon=1e-5)
        # rate = 12 (24)
        b2 = SepConv_BN(x, 256, 'aspp2',
                        rate=atrous_rates[1], depth_activation=True, epsilon=1e-5)
        # rate = 18 (36)
        b3 = SepConv_BN(x, 256, 'aspp3',
                        rate=atrous_rates[2], depth_activation=True, epsilon=1e-5)

        # concatenate ASPP branches & project
        x = Concatenate()([b4, b0, b1, b2, b3])
    else:
        x = Concatenate()([b4, b0])

    x = Conv2D(256, (1, 1), padding='same',
               use_bias=False, name='concat_projection')(x)
    x = BatchNormalization(name='concat_projection_BN', epsilon=1e-5)(x)
    x = Activation(tf.nn.relu)(x)
    x = Dropout(0.1)(x)
    # DeepLab v.3+ decoder

    if backbone == 'xception':
        # Feature projection
        # x4 (x2) block
        skip_size = tf.keras.backend.int_shape(skip1)
        x = tf.keras.layers.experimental.preprocessing.Resizing(
                *skip_size[1:3], interpolation="bilinear"
            )(x)
        dec_skip1 = Conv2D(48, (1, 1), padding='same',
                           use_bias=False, name='feature_projection0')(skip1)
        dec_skip1 = BatchNormalization(
            name='feature_projection0_BN', epsilon=1e-5)(dec_skip1)
        dec_skip1 = Activation(tf.nn.relu)(dec_skip1)
        x = Concatenate()([x, dec_skip1])
        x = SepConv_BN(x, 256, 'decoder_conv0',
                       depth_activation=True, epsilon=1e-5)
        x = SepConv_BN(x, 256, 'decoder_conv1',
                       depth_activation=True, epsilon=1e-5)

    # you can use it with arbitary number of classes
    if (weights == 'pascal_voc' and classes == 21) or (weights == 'cityscapes' and classes == 19):
        last_layer_name = 'logits_semantic'
    else:
        last_layer_name = 'custom_logits_semantic'

    x = Conv2D(classes, (1, 1), padding='same', name=last_layer_name)(x)
    size_before3 = tf.keras.backend.int_shape(img_input)
    x = tf.keras.layers.experimental.preprocessing.Resizing(
            *size_before3[1:3], interpolation="bilinear"
        )(x)
    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input

    if activation in {'softmax', 'sigmoid'}:
        x = tf.keras.layers.Activation(activation)(x)

    model = Model(inputs, x, name='deeplabv3plus')

    # load weights

    if weights == 'pascal_voc':
        if backbone == 'xception':
            weights_path = get_file('deeplabv3_xception_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH_X,
                                    cache_subdir='models')
        else:
            weights_path = get_file('deeplabv3_mobilenetv2_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH_MOBILE,
                                    cache_subdir='models')
        model.load_weights(weights_path, by_name=True)
    elif weights == 'cityscapes':
        if backbone == 'xception':
            weights_path = get_file('deeplabv3_xception_tf_dim_ordering_tf_kernels_cityscapes.h5',
                                    WEIGHTS_PATH_X_CS,
                                    cache_subdir='models')
        else:
            weights_path = get_file('deeplabv3_mobilenetv2_tf_dim_ordering_tf_kernels_cityscapes.h5',
                                    WEIGHTS_PATH_MOBILE_CS,
                                    cache_subdir='models')
        model.load_weights(weights_path, by_name=True)
    else:
        pass
    return model

In [ ]:
WEIGHTS_PATH_X = "https://github.com/bonlime/keras-deeplab-v3-plus/releases/download/1.1/deeplabv3_xception_tf_dim_ordering_tf_kernels.h5"
WEIGHTS_PATH_MOBILE = "https://github.com/bonlime/keras-deeplab-v3-plus/releases/download/1.1/deeplabv3_mobilenetv2_tf_dim_ordering_tf_kernels.h5"
WEIGHTS_PATH_X_CS = "https://github.com/bonlime/keras-deeplab-v3-plus/releases/download/1.2/deeplabv3_xception_tf_dim_ordering_tf_kernels_cityscapes.h5"
WEIGHTS_PATH_MOBILE_CS = "https://github.com/bonlime/keras-deeplab-v3-plus/releases/download/1.2/deeplabv3_mobilenetv2_tf_dim_ordering_tf_kernels_cityscapes.h5"

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.python.keras.utils.data_utils import get_file


In [ ]:
INPUT_SHAPE = (512,512,3)
LR = 0.005
EPOCHS = 200
#MODEL_PATH='/data/0818_deeplabv3_xception_pascal_hsv_tumor.hdf5'
MODEL_PATH = './model/0819_unet_inceptionv3_imagenet_hsv_tumor.hdf5'

In [ ]:
from tensorflow.keras.callbacks import Callback
# define optomizer
optim = Adam(INITIAL_LEARNING_RATE)

# Segmentation models losses can be combined together by '+' and scaled by integer or float factor
dice_loss = sm.losses.DiceLoss()

# default focal_loss (alpha=0.25, gamma=2.0)
focal_loss = sm.losses.BinaryFocalLoss(alpha=0.25, gamma=6.0)
total_loss = dice_loss + focal_loss

# actulally total_loss can be imported directly from library, above example just show you how to manipulate with losses
# total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss 

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]
#metrics = [my_metric]
class ClearMemory(Callback):
    def on_epoch_end(self,epoch,logs=None):
        gc.collect()
        k.clear_session()
        
mc = ModelCheckpoint(filepath=MODEL_PATH, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
es = EarlyStopping(monitor = 'val_loss',patience = 10)
lr = ReduceLROnPlateau(
    monitor='val_loss',  # 검증 손실을 기준으로 callback이 호출됩니다
    factor=0.5,          # callback 호출시 학습률을 1/2로 줄입니다
    patience=4,         # epoch 4 동안 개선되지 않으면 callback이 호출됩니다
)
callbacks_list = [mc,es,lr,ClearMemory()]
#mc = ModelCheckpoint(history_files); lr = ReduceLROnPlateau()
#callback_list = [mc,lr]

In [ ]:
from tensorflow.python.keras.utils.data_utils import get_file

In [ ]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
with strategy.scope():
    model = sm.Deeplabv3(weights = WEIGHT,input_shape = INPUT_SHAPE,classes=1,activation=ACTIVATION, backbone=BACKBONE,)
    #model = sm.Unet(backbone_name = 'inceptionv3',input_shape = INPUT_SHAPE,encoder_freeze=True)
    model.compile(optim, total_loss, metrics)

In [ ]:
model.fit(train_gen, 
          epochs = EPOCHS,
          validation_data=valid_gen,
          callbacks=callbacks_list,
          max_queue_size=20,
          workers=10,
          #use_multiprocessing=True,
          verbose=1
         )

In [ ]:
min(model.history.history['val_loss'])

In [ ]:
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')

plt.xlabel('Epoch')
plt.legend(['Train', 'Validate'], loc='upper left')
plt.show()

In [ ]:
e_val = model.evaluate(test_gen)
print(f'IOU Score : {e_val[1]:.2f} F1-Score : {e_val[2]:.2f}')

In [ ]:
import json

In [ ]:
model_summray = MODEL_PATH.split('/')[-1][:-5]
save_json = {'model':model_summray,'loss':e_val[0],'iou':e_val[1],'f1-score':e_val[2]}

with open(f'./model/eval_{model_summray}.json','w') as json_file:
    json.dump(save_json,json_file)
    

In [ ]:
batch_x,batch_y=  test_gen.__getitem__(5)
for i in range(10,20):

    img,msk = batch_x[i],batch_y[i]
    img = np.expand_dims(img,axis=0)
    pr_msk = model.predict(img)
    pr_msk = np.squeeze(pr_msk)
    pr_msk = np.where(pr_msk>0.5,pr_msk,0) # prob threhsold 0.5
    plt.figure(figsize = (8,6))
    plt.subplot(1,3,1); plt.title('predict'); plt.imshow(pr_msk)
    plt.subplot(1,3,2); plt.title('true_mask'); plt.imshow(np.squeeze(batch_y[i]))
    plt.subplot(1,3,3); plt.title('true_img'); plt.imshow(np.squeeze((255*img).astype(np.uint8)))

#pr_msk = np.expand_dims(pr_msk,axis=-1)
